# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt
# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Dsc
0,0,Maragogi,BR,-9.0122,-35.2225,76.95,83,74,8.41,broken clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,63.18,94,100,6.60,overcast clouds
2,2,Zaozerne,UA,45.1582,33.2787,54.37,82,100,24.56,overcast clouds
3,3,Turek,PL,52.0155,18.5005,30.61,77,7,7.00,clear sky
4,4,Saint-Philippe,RE,-21.3585,55.7679,69.39,71,37,6.58,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temprature criteria?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is maximum temprature criteria?"))

What is the minimum temprature criteria?60
What is maximum temprature criteria?90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Dsc
0,0,Maragogi,BR,-9.0122,-35.2225,76.95,83,74,8.41,broken clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,63.18,94,100,6.60,overcast clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,69.39,71,37,6.58,scattered clouds
5,5,Kruisfontein,ZA,-34.0033,24.7314,61.88,69,100,2.35,overcast clouds
6,6,Rafai,CF,4.9500,23.9167,69.19,83,9,2.08,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Dsc
0,0,Maragogi,BR,-9.0122,-35.2225,76.95,83,74,8.41,broken clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,63.18,94,100,6.60,overcast clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,69.39,71,37,6.58,scattered clouds
5,5,Kruisfontein,ZA,-34.0033,24.7314,61.88,69,100,2.35,overcast clouds
6,6,Rafai,CF,4.9500,23.9167,69.19,83,9,2.08,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country","Max Temp","Weather Dsc","Lat","Lng"]].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather Dsc,Lat,Lng
0,Maragogi,BR,76.95,broken clouds,-9.0122,-35.2225
1,Port Alfred,ZA,63.18,overcast clouds,-33.5906,26.8910
4,Saint-Philippe,RE,69.39,scattered clouds,-21.3585,55.7679
5,Kruisfontein,ZA,61.88,overcast clouds,-34.0033,24.7314
6,Rafai,CF,69.19,clear sky,4.9500,23.9167


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather Dsc,Lat,Lng,Hotel Name
0,Maragogi,BR,76.95,broken clouds,-9.0122,-35.2225,
1,Port Alfred,ZA,63.18,overcast clouds,-33.5906,26.8910,
4,Saint-Philippe,RE,69.39,scattered clouds,-21.3585,55.7679,
5,Kruisfontein,ZA,61.88,overcast clouds,-34.0033,24.7314,
6,Rafai,CF,69.19,clear sky,4.9500,23.9167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather Dsc,Lat,Lng,Hotel Name
0,Maragogi,BR,76.95,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
1,Port Alfred,ZA,63.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel & Spa
4,Saint-Philippe,RE,69.39,scattered clouds,-21.3585,55.7679,Le four à pain
5,Kruisfontein,ZA,61.88,overcast clouds,-34.0033,24.7314,Oyster Bay House Rental
6,Rafai,CF,69.19,clear sky,4.9500,23.9167,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Weather Dsc,Lat,Lng,Hotel Name
0,Maragogi,BR,76.95,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
1,Port Alfred,ZA,63.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel & Spa
4,Saint-Philippe,RE,69.39,scattered clouds,-21.3585,55.7679,Le four à pain
5,Kruisfontein,ZA,61.88,overcast clouds,-34.0033,24.7314,Oyster Bay House Rental
6,Rafai,CF,69.19,clear sky,4.9500,23.9167,
...,...,...,...,...,...,...,...
681,Gurupi,BR,81.39,broken clouds,-11.7292,-49.0686,Via Norte Hotel
682,Valparaiso,CL,66.47,few clouds,-33.0393,-71.6273,Hotel Ultramar
683,Abengourou,CI,68.94,scattered clouds,6.7297,-3.4964,Hotel des Princes
684,Goya,AR,80.29,clear sky,-29.1400,-59.2626,Apart Costanera Goya


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
#clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Dsc} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Salinas do Maragogi All Inclusive Resort</dd>\n<dt>City</dt><dd>Maragogi</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>broken clouds and 76.95 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The Halyards Hotel & Spa</dd>\n<dt>City</dt><dd>Port Alfred</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 63.18 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Le four à pain</dd>\n<dt>City</dt><dd>Saint-Philippe</dd>\n<dt>Country</dt><dd>RE</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 69.39 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Oyster Bay House Rental</dd>\n<dt>City</dt><dd>Kruisfontein</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 61.88 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Rafai</dd>\n<dt>Country</dt><dd>CF</dd>\n<dt>Current Weather</dt><dd>clear sky and 69.19 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-9.0122,-35.2225
1,-33.5906,26.8910
4,-21.3585,55.7679
5,-34.0033,24.7314
6,4.9500,23.9167
7,0.5333,72.9333
8,17.3333,42.6667
9,-21.5667,21.7833
11,-14.7972,-57.2881
12,-0.6000,73.0833


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
# YOUR CODE HERE
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
# YOUR CODE HERE
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
# YOUR CODE HERE
fig